In [ ]:
import os
%pip install langchain
from langchain import HuggingFaceHub

In [ ]:
import pandas as pd
df = pd.read_csv("jobs (1).csv")
df["result"]=df["company_id"]+df["job_name"] +	df["taglist"]	+df["location"]	+df["three_reasons"]	+df["description"]

In [ ]:
df.sample(5)

In [ ]:
!pip install faiss-cpu

In [ ]:
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer

In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer
import google.generativeai as genai
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model = SentenceTransformer(model_name)
api_key = "YOUR_GEN_AI_API_KEY"
genai.configure(api_key=api_key)
genmodel = genai.GenerativeModel('gemini-pro')
import faiss
import numpy as np
chunks_embeddings = []
for i in range(len(df)):
    if isinstance(df['result'][i], str):
        # Process only if the element is a string
        text = df["result"][i]
        embedding = model.encode(text)
        chunks_embeddings.append((text, embedding))
    else:
        # Skip if the element is not a string
        continue

dimension = chunks_embeddings[0][1].shape[0]
embeddings_array = np.array([emb for _, emb in chunks_embeddings]).astype('float32')
faiss_index = faiss.IndexFlatL2(dimension)
faiss_index.add(embeddings_array)
import pickle
with open("pickle_file", 'wb') as f:
    pickle.dump(faiss_index, f)

def answer(text):
  query_embedding = model.encode(text)
  query_embedding = np.array([query_embedding]).astype('float32')
  k = 10
  D, I = faiss_index.search(query_embedding, k)
  retrieved_list = [chunks_embeddings[i][0] for i in I[0]]
  import re
  texts=[]
  for text in retrieved_list:
   texts.append(text)
  response = genmodel.generate_content("find the name of the company , job , description and location for each item in the list of individually {texts}".format(texts=texts))

  return response.text

# Get user input
text = input("Enter the details here: ")
response = answer(text)
print(response)



In [ ]:
with open("pickle_file", 'rb') as f:
    loaded_data = pickle.load(f)

In [ ]:
def answer(text):
  query_embedding = model.encode(text)
  query_embedding = np.array([query_embedding]).astype('float32')
  k = 10
  D, I = loaded_data.search(query_embedding, k)
  retrieved_list = [chunks_embeddings[i][0] for i in I[0]]
  import re
  texts=[]
  for text in retrieved_list:
   texts.append(text)
  response = genmodel.generate_content("find the name of the company , job , description and location for all the 10 elements in the  {texts}".format(texts=texts))
  return response.text


text = input("Enter the details here: ")
response = answer(text)
print(response)